In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import torch
import scipy.sparse as sp
import glob
import warnings
import anndata
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import seaborn as sns
import torch
import pyro

In [4]:
import spaceoracle
from spaceoracle import SpaceOracle

In [5]:
adata_train = sc.read_h5ad('./.cache/adata_train.h5ad')
adata_train

AnnData object with n_obs × n_vars = 11567 × 5013
    obs: 'cluster', 'rctd_cluster', 'rctd_celltypes'
    uns: 'log1p'
    obsm: 'X_spatial', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'

In [6]:
import shutil
import os
    
models_dir = './models'
if os.path.exists(models_dir):
    shutil.rmtree(models_dir)
os.makedirs(models_dir)

In [7]:
sc.pp.highly_variable_genes(adata_train, flavor="seurat", n_top_genes=1000, subset=True)
adata_train

AnnData object with n_obs × n_vars = 11567 × 1000
    obs: 'cluster', 'rctd_cluster', 'rctd_celltypes'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'X_spatial', 'rctd_results', 'spatial'
    layers: 'imputed_count', 'normalized_count', 'raw_count'

In [8]:
so = spaceoracle.SpaceOracle(
    adata=adata_train,
    annot='rctd_cluster', 
    max_epochs=1, 
    learning_rate=1e-3, 
    spatial_dim=64,
    batch_size=512,
    threshold_lambda=3e3,
    test_mode=True
)

In [9]:
so.run()

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Fitting Cd34 with 3 modulators
	3 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
0: -20.8318 | -0.0000
1: -61.2111 | -0.0000
2: -356.3924 | -0.0000
3: -42.2809 | -0.0000
4: -109.5530 | -0.0000
5: -22.2838 | -0.0000
6: -19.4240 | -0.0000
Fitting Igkv14-126 with 1 modulators
	1 Transcription Factors
	0 Ligand-Receptor Pairs
	0 TranscriptionFactor-Ligand Pairs
0: -4.2254 | -0.0000
1: -36.3739 | -0.0000
2: -47.8657 | -0.0000
3: -40.3286 | -0.0000
4: -35.5016 | -0.0000
5: -26.9966 | -0.0000
6: -35.9078 | -0.0000
